In [1]:
import pandas as pd
import numpy as np
import requests
import certifi
from pyevmasm import instruction_tables, disassemble_hex, disassemble_all, assemble_hex
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
addresses = pd.read_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/addresses.csv")
left_ds = pd.read_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/Dataset/dataset.csv")

In [3]:
addr_list = []
bytecode_list = {}
opcode_list = {}

In [4]:
for addr in addresses['address']: #creo una lista con tutti gli address
      addr_list.append(addr)

In [5]:
for addr in addr_list: # ciclo su tutti gli indirizzi presenti nella lista
    normal_tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=" + addr + \
                   "&startblock=0&endblock=99999999&page=1&offset=10000&sort=asc&apikey=SAKUKN5G8E7R2IYIWI2IVS7ZRKXDAX8NE6"
    response_normal = requests.get(normal_tx_url, verify = certifi.where()) # la get restituisce un Response obj
    address_content_normal = response_normal.json() # la risposta e' in formato json, si ottiene un dizionario python
    result_normal = address_content_normal.get('result') # facendo la get e si ottiene tutto cio' che sta dopo result, si ottiene una lista
    
    for t in result_normal:
        if (t['to'] == ''): # prendo l'oggetto json con la creazione del contratto che ha il to = ''
            bytecode = (t['input']) # aggiungo il valore della chiave input che è uguale al bytecode
            bytecode_list[addr]=bytecode # faccio append con l'obiettivo di creare una lista con tutti i bytecode.

In [6]:
# creo una lista di bytecode
for k in bytecode_list.keys():
    opcode = disassemble_hex(bytecode_list[k])# conversione bytecode in opcode
    opcode_list[k] = opcode

In [7]:
df_full_comm = pd.DataFrame()
for addr in opcode_list.keys():
    row = []
    list_comm = pd.Series(opcode_list[addr].split('\n')) 
    list_comm = list_comm.str.replace("PUSH.*", "PUSH")
    list_comm = list_comm.str.replace("DUP.*", "DUP")
    list_comm = list_comm.str.replace("SWAP.*", "SWAP")
    list_comm = list_comm.str.replace("LOG.*", "LOG")
    df_comm = pd.DataFrame(list_comm.value_counts())
    df_comm = df_comm.T
    df_comm['address'] = [addr]
    df_full_comm = pd.concat([df_full_comm,df_comm] )

In [8]:
#df_full_comm

In [9]:
#df_full_comm.to_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/bytecode_decode.csv" , index=False)

In [10]:
df_full = pd.merge(left_ds, df_full_comm, how='left', on = 'address')
df_full.to_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/bytecode_decode.csv" , index=False)